## ETL Dados Novos (2023-2025)

### 4.1. Justificativa Técnica e Abordagem Profissional

O pipeline ETL para dados recentes (2023-2025) foi desenvolvido com foco em **flexibilidade e resiliência**, características essenciais para lidar com dados governamentais em evolução.

| Desafio | Solução Profissional Implementada | Justificativa |
| :--- | :--- | :--- |
| **Formato Variável** | Leitura flexível com múltiplos separadores e encodings. | **Adaptabilidade:**  Permite processar arquivos com diferentes padrões de formatação sem intervenção manual |
| **Inconsistências Estruturais** |	Detecção e correção automática de colunas mescladas. | **Resiliência:** Soluciona problemas comuns de exportação onde dados são concatenados em uma única coluna|
| **Tipagem Inadequada** | Conversão robusta de tipos numéricos e datas	Integridade. | **Analítica:** Garante que cálculos e agregações funcionem corretamente |
| **Evolução do Schema** |	Processamento individual por ano com consolidação final. | **Manutenibilidade:** Permite tratar variações anuais no schema antes da unificação |

---
### 4.2. Etapas do ETL (Pipeline Principal)

A fase de *Preparação dos Dados** segue uma abordagem em camadas, orquestrada pelo método consolidar_todos_anos().

| **Etapa** | **Método** | **Justificativa Técnica** |
| :--- | :--- | :--- |
| **1. Descoberta** | `listar_arquivos_raw()` |	Identifica automaticamente arquivos relevantes, excluindo anos já tratados separadamente. |
| **2. Leitura Adaptativa** | `_ler_arquivo_flexivel()` | Tentativas sequenciais com diferentes configurações de separador e encoding. |
| **3. Correção Estrutural** |	`_corrigir_problemas_especificos()` | Divide colunas mescladas e aplica limpeza básica de nomes. |
| **4. Normalização de Tipos** | `_corrigir_problemas_especificos()`| Converte numéricos, datas e flags para formatos analiticamente úteis. |
| **5. Enriquecimento** | `processar_arquivo_individual()`| Adiciona coluna ano_compra para rastreabilidade temporal. |
| **6. Consolidação** | `consolidar_todos_anos()`| Unifica todos os anos em dataset único com ordenação temporal. |

---
### 4.3. Benefícios e Implicação para o Projeto de Análise de Gestão de Preços e Gestão da Demanda

O pipeline ETL para dados recentes estabelece as bases para análises temporais robustas e modelagem preditiva:

 * **Série Temporal Contínua:**  A consolidação de 2023-2025 com os dados antigos (2020-2022) permite análise de tendências em 6 anos de compras públicas

 * **Qualidade de Dados Garantida:** O tratamento automático de inconsistências assegura confiabilidade nas análises financeiras e de volume

 * **Preparação para Machine Learning:** Dados limpos e tipados corretamente são essenciais para treinar modelos de previsão de demanda

 * **Escalabilidade:** A arquitetura modular permite fácil inclusão de novos anos sem refatoração

### 4.4. Alinhamento com CRISP-DM

 * **Business Understanding:** Necessidade de analisar padrões recentes de compras para otimização orçamentária

 * **Data Understanding:** Identificação de variações anuais no formato dos dados do OpenDataSUS

 * **Data Preparation:** Este pipeline - transformação de dados brutos em formato analítico

 * **Modeling:** Base preparada para modelagem dimensional e análises estatísticas

 * **Evaluation:** Validação através de checagens de integridade e completude

 * **Deployment:** Processo reprodutível para atualizações anuais dos dados

### 4.5. Saída Esperada

O pipeline produz um dataset consolidado com:

 - Schema padronizado entre todos os anos

 - Tipos de dados corretos para análise

 - Metadados temporais completos

 - Indicadores de qualidade incorporados

In [2]:
import pandas as pd
import os
import re

class ETLComprasPublicas:
    
    def __init__(self, pasta_raw):
        # Usa o caminho exato que você forneceu
        self.pasta_raw = pasta_raw
        self.pasta_base = os.path.dirname(os.path.dirname(pasta_raw))
        self.pasta_dados = os.path.join(self.pasta_base, "data")
        self.pasta_processed = os.path.join(self.pasta_dados, "processed")
        
        print(f" Estrutura de pastas:")
        print(f"• Raw: {self.pasta_raw}")
        print(f"• Processed: {self.pasta_processed}")
        
        # Garante que as pastas existem
        os.makedirs(self.pasta_raw, exist_ok=True)
        os.makedirs(self.pasta_processed, exist_ok=True)

    def listar_arquivos_raw(self):
        """Lista arquivos CSV excluindo anos antigos (2020-2022)"""
        if not os.path.exists(self.pasta_raw):
            print(f" Pasta raw não existe: {self.pasta_raw}")
            return []
            
        arquivos = [os.path.join(self.pasta_raw, f) for f in os.listdir(self.pasta_raw)
                    if f.endswith('.csv') and not any(ano in f for ano in ['2020', '2021', '2022'])]
        
        print(f" Arquivos encontrados: {[os.path.basename(f) for f in arquivos]}")
        return arquivos

    def extrair_ano_do_arquivo(self, caminho_arquivo):
        """Extrai o ano do nome do arquivo"""
        nome = os.path.basename(caminho_arquivo)
        match = re.search(r'20\d{2}', nome)
        return int(match.group()) if match else None

    def _ler_arquivo_flexivel(self, caminho_arquivo):
        """Leitura flexível com diferentes separadores e encodings"""
        if not os.path.exists(caminho_arquivo):
            print(f" Arquivo não encontrado: {caminho_arquivo}")
            return None

        tentativas = [
            {'sep': ',', 'encoding': 'utf-8'},
            {'sep': ';', 'encoding': 'utf-8'},
            {'sep': '\t', 'encoding': 'utf-8'},
            {'sep': '\t', 'encoding': 'latin-1'},
        ]

        for config in tentativas:
            try:
                df = pd.read_csv(caminho_arquivo, sep=config['sep'], encoding=config['encoding'], low_memory=False)
                print(f" Arquivo lido com sep='{config['sep']}' e encoding='{config['encoding']}'")
                return df
            except Exception:
                continue

        print(f" Todas as tentativas de leitura falharam para: {caminho_arquivo}")
        return None

    def _corrigir_problemas_especificos(self, df):
        """Aplica correções específicas ao DataFrame"""
        if df is None or len(df) == 0:
            return df

        print(" Aplicando correções específicas...")

        # Limpeza dos nomes das colunas
        df.columns = [col.strip() for col in df.columns]

        # Divisão de coluna única (se necessário)
        if len(df.columns) == 1 and ';' in str(df.iloc[0, 0]):
            colunas_divididas = df.iloc[:, 0].str.split(';', expand=True)
            novo_cabecalho = colunas_divididas.iloc[0]
            df = colunas_divididas[1:]
            df.columns = novo_cabecalho
            df.reset_index(drop=True, inplace=True)
            print(f" Colunas divididas: {len(df.columns)}")

        # Corrigir valores numéricos
        colunas_numericas = ['qtd_itens_comprados', 'preco_unitario', 'preco_total', 'capacidade']
        for coluna in colunas_numericas:
            if coluna in df.columns:
                df[coluna] = pd.to_numeric(df[coluna], errors='coerce').fillna(0)

        # Corrigir datas
        colunas_data = ['compra', 'insercao']
        for coluna in colunas_data:
            if coluna in df.columns:
                df[coluna] = pd.to_datetime(df[coluna], errors='coerce')

        # Corrigir flags genéricos
        if 'generico' in df.columns:
            df['generico'] = (
                df['generico']
                .astype(str)
                .str.strip()
                .str.upper()
                .map({'S': 'SIM', 'N': 'NÃO'})
                .fillna('NÃO')
            )

        print(" Correções aplicadas com sucesso")
        return df

    def processar_arquivo_individual(self, caminho_arquivo, forcar_reprocessamento=False):
        """Processa um único arquivo CSV"""
        ano = self.extrair_ano_do_arquivo(caminho_arquivo)
        nome_arquivo = os.path.basename(caminho_arquivo)
        
        print(f"\n Processando: {nome_arquivo}")

        # 1. Leitura flexível
        df = self._ler_arquivo_flexivel(caminho_arquivo)
        if df is None or df.empty:
            print(f" Falha ao ler arquivo: {nome_arquivo}")
            return None

        print(f" Arquivo lido: {len(df)} registros, {len(df.columns)} colunas")

        # 2. Correções específicas
        df_tratado = self._corrigir_problemas_especificos(df)
        if df_tratado is None or df_tratado.empty:
            print(f" Tratamento falhou para: {nome_arquivo}")
            return None

        # 3. Garante coluna de ano
        if 'ano_compra' not in df_tratado.columns:
            df_tratado['ano_compra'] = ano

        print(f" Processamento concluído: {nome_arquivo} ({len(df_tratado)} registros)")
        return df_tratado

    def consolidar_todos_anos(self, forcar_reprocessamento=False):
        """Consolida todos os anos processados em um único DataFrame"""
        print(" Iniciando consolidação de todos os anos...")

        arquivos = self.listar_arquivos_raw()
        if not arquivos:
            print(" Nenhum arquivo CSV encontrado.")
            print(f" Coloque os arquivos .csv (2023.csv, 2024.csv, 2025.csv) em: {self.pasta_raw}")
            return None

        todos_dados = []
        anos_processados = []

        for arquivo in arquivos:
            df_ano = self.processar_arquivo_individual(arquivo, forcar_reprocessamento)

            if df_ano is None or df_ano.empty:
                print(f" Ignorado: {os.path.basename(arquivo)}")
                continue

            # Adiciona ao consolidado
            todos_dados.append(df_ano)
            ano = self.extrair_ano_do_arquivo(arquivo)
            anos_processados.append(ano)
            print(f" {ano}: {len(df_ano):,} registros processados")

        if not todos_dados:
            print(" Nenhum dado foi processado com sucesso.")
            return None

        # Consolida os dados
        df_consolidado = pd.concat(todos_dados, ignore_index=True)

        # Ordena por data de compra
        if 'compra' in df_consolidado.columns:
            df_consolidado['compra'] = pd.to_datetime(df_consolidado['compra'], errors='coerce')
            df_consolidado = df_consolidado.sort_values('compra')

        print("\n Consolidação completa!")
        print(f" Total de registros: {len(df_consolidado):,}")
        print(f" Anos processados: {sorted(set(anos_processados))}")
        
        return df_consolidado

# --- EXECUÇÃO NO NOTEBOOK ---
print("=" * 60)
print(" ETL - COMPRAS PÚBLICAS DE MEDICAMENTOS (2023-2025)")
print("=" * 60)

# Instanciar e executar o ETL com o caminho exato
caminho_exato = r"C:\Users\debor\OneDrive\Github\compras_medicamentos_sus\data\raw"
etl = ETLComprasPublicas(caminho_exato)
df_final = etl.consolidar_todos_anos(forcar_reprocessamento=True)

if df_final is not None and not df_final.empty:
    print(f"\n" + "="*50)
    print(" PROCESSAMENTO COMPLETO!")
    print("="*50)
    print(f" Total de registros: {len(df_final):,}")
    
    # Exibir schema e head
    print(f"\n SCHEMA DO DATASET:")
    print(f"• Dimensões: {df_final.shape[0]} linhas x {df_final.shape[1]} colunas")
    print(f"• Colunas: {list(df_final.columns)}")
    
    print(f"\n AMOSTRA DOS DADOS (primeiras 5 linhas):")
    display(df_final.head())
    
    # Estatísticas básicas
    print(f"\n ESTATÍSTICAS BÁSICAS:")
    if 'preco_total' in df_final.columns:
        total_gasto = df_final['preco_total'].sum()
        print(f"• Total gasto: R$ {total_gasto:,.2f}")
    if 'uf' in df_final.columns:
        print(f"• Estados participantes: {df_final['uf'].nunique()}")
    if 'municipio_instituicao' in df_final.columns:
        print(f"• Municípios: {df_final['municipio_instituicao'].nunique()}")
    if 'descricao_catmat' in df_final.columns:
        print(f"• Medicamentos diferentes: {df_final['descricao_catmat'].nunique()}")
        
    # Gastos por ano
    if 'ano_compra' in df_final.columns and 'preco_total' in df_final.columns:
        print(f"\n GASTOS POR ANO:")
        gastos_por_ano = df_final.groupby('ano_compra')['preco_total'].sum()
        for ano, gasto in gastos_por_ano.items():
            print(f"  {ano}: R$ {gasto:,.2f}")
            
else:
    print("\n FALHA NO PROCESSAMENTO.")
    print("Verifique se os arquivos CSV estão na pasta correta.")

 ETL - COMPRAS PÚBLICAS DE MEDICAMENTOS (2023-2025)
 Estrutura de pastas:
• Raw: C:\Users\debor\OneDrive\Github\compras_medicamentos_sus\data\raw
• Processed: C:\Users\debor\OneDrive\Github\compras_medicamentos_sus\data\processed
 Iniciando consolidação de todos os anos...
 Arquivos encontrados: ['2023.csv', '2024.csv', '2025.csv']

 Processando: 2023.csv
 Arquivo lido com sep=';' e encoding='utf-8'
 Arquivo lido: 29428 registros, 24 colunas
 Aplicando correções específicas...
 Correções aplicadas com sucesso
 Processamento concluído: 2023.csv (29428 registros)
 2023: 29,428 registros processados

 Processando: 2024.csv
 Arquivo lido com sep=';' e encoding='utf-8'
 Arquivo lido: 20512 registros, 24 colunas
 Aplicando correções específicas...
 Correções aplicadas com sucesso
 Processamento concluído: 2024.csv (20512 registros)
 2024: 20,512 registros processados

 Processando: 2025.csv
 Arquivo lido com sep=';' e encoding='utf-8'
 Arquivo lido: 2474 registros, 24 colunas
 Aplicando corr

,ano_compra,nome_instituicao,cnpj_instituicao,municipio_instituicao,uf,compra,insercao,codigo_br,descricao_catmat,unidade_fornecimento,...,capacidade,unidade_medida,unidade_fornecimento_capacidade,cnpj_fornecedor,fornecedor,cnpj_fabricante,fabricante,qtd_itens_comprados,preco_unitario,preco_total
0,2023,FUNDO MUNICIPAL DE SAUDE DE MARABA,18478187000107,MARABA,PA,2023-01-01,2024-03-15,327534,"CORANTE, TIPO:CONJUNTO REAGENTE PARA COLORAÇÃO...",FRASCO,...,1.0,L,FRASCO 1.00 L,41733464000194,SHL - SAUDE HOSPITALAR E LABORATORIAL LTDA,562583000144,RENYLAB - QUIMICA E FARMACEUTICA LTDA,10,67.89,678.90
19,2023,FUNDO MUNICIPAL DE SAUDE DE MARABA,18478187000107,MARABA,PA,2023-01-01,2024-03-15,423975,"PIPETA, TIPO:PASTEUR, CAPACIDADE:3 ML, MATERIA...",UNIDADE,...,0.0,NaN,UNIDADE,5323167000107,CIRUBEL COMERCIO E REPRESENTACOES DE PRODUTOS ...,57893521000132,PERFITECNICA PERFIS TECNICOS DE BORRACHA LTDA,36,45.00,1620.00
18,2023,FUNDO MUNICIPAL DE SAUDE DE MARABA,18478187000107,MARABA,PA,2023-01-01,2024-03-15,395610,"REAGENTE ANALÍTICO 4, TIPO:ÁLCOOL-ÁCIDO, CONCE...",UNIDADE,...,0.0,NaN,UNIDADE,41733464000194,SHL - SAUDE HOSPITALAR E LABORATORIAL LTDA,562583000144,RENYLAB - QUIMICA E FARMACEUTICA LTDA,55,48.49,2666.95
17,2023,FUNDO MUNICIPAL DE SAUDE DE MARABA,18478187000107,MARABA,PA,2023-01-01,2024-03-15,380528,"REAGENTE ANALÍTICO 4, TIPO:ÁLCOOL-ÁCIDO, CONCE...",UNIDADE,...,0.0,NaN,UNIDADE,7944100000115,PROC9 INDUSTRIA QUIMICA LTDA,7944100000115,PROC9 INDUSTRIA QUIMICA LTDA,7,24.00,168.00
16,2023,FUNDO MUNICIPAL DE SAUDE DE MARABA,18478187000107,MARABA,PA,2023-01-01,2024-03-15,457503,"CORANTE, TIPO :PARDO DE BISMARCK, CARACTERÍSTI...",UNIDADE,...,0.0,NaN,UNIDADE,7944100000115,PROC9 INDUSTRIA QUIMICA LTDA,7944100000115,PROC9 INDUSTRIA QUIMICA LTDA,4,190.00,760.00



 ESTATÍSTICAS BÁSICAS:
• Total gasto: R$ 36,765,122,621.47
• Estados participantes: 21
• Municípios: 313
• Medicamentos diferentes: 6094

 GASTOS POR ANO:
  2023: R$ 7,089,089,929.44
  2024: R$ 6,459,384,424.21
  2025: R$ 23,216,648,267.83
